<a href="https://colab.research.google.com/github/Anjalichimnani/EVA4/blob/master/Assignment_4/EVA4_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1) #input -? OUtput? RF
        self.batchn1 = nn.BatchNorm2d(8)
        self.dropout = nn.Dropout2d(0.075)

        self.conv2 = nn.Conv2d(8, 16, 3, padding=1)
        self.batchn2 = nn.BatchNorm2d(16)

        self.conv3 = nn.Conv2d(16, 24, 3, padding=1)
        self.batchn3 = nn.BatchNorm2d(24)

        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv21 = nn.Conv2d(24, 12, 1)
        self.batchn21 = nn.BatchNorm2d(12)

        self.conv4 = nn.Conv2d(12, 12, 3, padding=1)
        self.batchn4 = nn.BatchNorm2d(12)
        self.conv5 = nn.Conv2d(12, 16, 3, padding=1)
        self.batchn5 = nn.BatchNorm2d(16)
        
        self.pool2 = nn.MaxPool2d(2, 2)
        #self.conv41 = nn.Conv2d(24, 8, 1)
        #self.batchn41 = nn.BatchNorm2d(8)

        self.conv6 = nn.Conv2d(16, 16, 3)
        self.batchn6 = nn.BatchNorm2d(16)

        self.conv7 = nn.Conv2d(16, 20, 3)
        self.batchn7 = nn.BatchNorm2d(20)

        self.conv8 = nn.Conv2d(20, 10, 3)

    def forward(self, x):
        x = F.relu(self.dropout(self.batchn2(self.conv2(F.relu(self.batchn1(self.conv1(x)))))))
        x = F.relu(self.dropout(self.batchn21(self.conv21(self.pool1(F.relu(self.dropout(self.batchn3(self.conv3(x)))))))))
        x = self.pool2(F.relu(self.dropout(self.batchn5(self.conv5(F.relu(self.dropout(self.batchn4(self.conv4(x)))))))))
        x = F.relu(self.dropout(self.batchn7(self.conv7(F.relu(self.dropout(self.batchn6(self.conv6(x))))))))
        x = self.conv8(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [30]:
!pip install torchsummary
!pip install tqdm -U

from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Requirement already up-to-date: tqdm in /usr/local/lib/python3.6/dist-packages (4.43.0)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
            Conv2d-3           [-1, 16, 28, 28]           1,168
       BatchNorm2d-4           [-1, 16, 28, 28]              32
         Dropout2d-5           [-1, 16, 28, 28]               0
            Conv2d-6           [-1, 24, 28, 28]           3,480
       BatchNorm2d-7           [-1, 24, 28, 28]              48
         Dropout2d-8           [-1, 24, 28, 28]               0
         MaxPool2d-9           [-1, 24, 14, 14]               0
           Conv2d-10           [-1, 12, 14, 14]             300
      BatchNorm2d-11           [-1, 12, 14, 14]              24
        Dropout2d-12           [-1, 12, 14, 14]               0
           Conv

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm.notebook import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    train_loss = 0
    correct = 0
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)

        loss = F.nll_loss(output, target)
        loss.backward()
        train_loss += F.nll_loss(output, target, reduction = 'sum').item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')

    train_loss /= len(train_loader.dataset)

    print ('\n\nTrain Set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        train_loss, correct, len(train_loader.dataset), 100. * correct/len(train_loader.dataset)
    ))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [36]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    print ('\nEpoch: ', epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


Epoch:  1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.





Train Set: Average Loss: 0.3115, Accuracy: 54502/60000 (90.84%)


Test set: Average loss: 0.0579, Accuracy: 9830/10000 (98.30%)


Epoch:  2





Train Set: Average Loss: 0.0884, Accuracy: 58408/60000 (97.35%)


Test set: Average loss: 0.0426, Accuracy: 9864/10000 (98.64%)


Epoch:  3





Train Set: Average Loss: 0.0703, Accuracy: 58725/60000 (97.88%)


Test set: Average loss: 0.0349, Accuracy: 9885/10000 (98.85%)


Epoch:  4


...




Train Set: Average Loss: 0.0305, Accuracy: 59430/60000 (99.05%)


Test set: Average loss: 0.0202, Accuracy: 9936/10000 (99.36%)


Epoch:  16





Train Set: Average Loss: 0.0286, Accuracy: 59447/60000 (99.08%)


Test set: Average loss: 0.0201, Accuracy: 9934/10000 (99.34%)


Epoch:  17





Train Set: Average Loss: 0.0283, Accuracy: 59461/60000 (99.10%)


Test set: Average loss: 0.0197, Accuracy: 9936/10000 (99.36%)


Epoch:  18





Train Set: Average Loss: 0.0294, Accuracy: 59435/60000 (99.06%)


Test set: Average loss: 0.0190, Accuracy: 9941/10000 (99.41%)


Epoch:  19





Train Set: Average Loss: 0.0269, Accuracy: 59480/60000 (99.13%)


Test set: Average loss: 0.0209, Accuracy: 9939/10000 (99.39%)

